# Xiaoyue Zhu ICE 5

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [2]:
# IRIS is included in sci-kit learn. We can directly load it.
# See documentation here: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html
irisRaw = datasets.load_iris()

# Convert it to a pandas dataframe for better visualization
iris = pd.DataFrame(data=irisRaw.data, columns=irisRaw.feature_names)
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


# MCAR

In [3]:
iris_MCAR = iris.copy()

In [4]:
# Set the missingness rate at 30%
missing = 0.3

# Only introduce missing rate to the `sepal length (cm)`.
iris_MCAR.loc[iris_MCAR.sample(frac=missing).index, 'sepal length (cm)'] = np.nan

iris_MCAR

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,NaN,2.5,5.0,1.9
147,NaN,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [5]:
# Alternatively, iterate through all columns and introduce 20% missing to all variables.
for col in iris_MCAR.columns:
    iris_MCAR.loc[iris_MCAR.sample(frac=missing).index, col] = np.nan

# MAR

In [9]:
iris_MAR = iris.copy()

In [10]:
# Set the missingness rate at 30%
missing = 0.3

# Only introduce missing rate to the `sepal length (cm)`.
iris_MAR.loc[iris_MAR.sample(frac=missing, weights = 'sepal width (cm)').index, 'sepal length (cm)'] = np.nan

iris_MAR

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,NaN,3.4,5.4,2.3


# Listwise deletion

In [12]:
iris_MCAR_listwise = iris_MCAR.copy()
iris_MCAR_listwise.dropna()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
3,4.6,3.1,1.5,0.2
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
11,4.8,3.4,1.6,0.2
12,4.8,3.0,1.4,0.1
32,5.2,4.1,1.5,0.1
35,5.0,3.2,1.2,0.2
36,5.5,3.5,1.3,0.2
48,5.3,3.7,1.5,0.2


# Pairwise deletion

In [13]:
iris_MCAR_pairwise = iris_MCAR.copy()
iris_MAR_pairwise = iris_MAR.copy()

print("Mean estimation in MCAR data with pairwise deletion",
      iris_MCAR_pairwise['sepal length (cm)'].mean(), 
     "\nSD estimation in MCAR data with pairwise deletion",
      iris_MCAR_pairwise['sepal length (cm)'].std()
     )
print("Mean estimation in MAR data with pairwise deletion",
      iris_MAR_pairwise['sepal length (cm)'].mean(), 
     "\nSD estimation in MAR data with pairwise deletion",
      iris_MAR_pairwise['sepal length (cm)'].std()
     )
print("Mean estimation in the original data",
      iris['sepal length (cm)'].mean(),
      "\nSD estimation in the original data",
      iris['sepal length (cm)'].std()
     )

Mean estimation in MCAR data with pairwise deletion 5.680555555555554 
SD estimation in MCAR data with pairwise deletion 0.7563140581905038
Mean estimation in MAR data with pairwise deletion 5.8771428571428554 
SD estimation in MAR data with pairwise deletion 0.8759892367244836
Mean estimation in the original data 5.843333333333335 
SD estimation in the original data 0.8280661279778629


# Simple imputation

In [14]:
from sklearn.impute import SimpleImputer
iris_MAR_mean = iris_MAR.copy()

In [15]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(iris_MAR)
print(imp.transform(iris_MAR))

[[5.1        3.5        1.4        0.2       ]
 [4.9        3.         1.4        0.2       ]
 [4.7        3.2        1.3        0.2       ]
 [4.6        3.1        1.5        0.2       ]
 [5.         3.6        1.4        0.2       ]
 [5.87714286 3.9        1.7        0.4       ]
 [4.6        3.4        1.4        0.3       ]
 [5.         3.4        1.5        0.2       ]
 [5.87714286 2.9        1.4        0.2       ]
 [5.87714286 3.1        1.5        0.1       ]
 [5.87714286 3.7        1.5        0.2       ]
 [4.8        3.4        1.6        0.2       ]
 [4.8        3.         1.4        0.1       ]
 [4.3        3.         1.1        0.1       ]
 [5.8        4.         1.2        0.2       ]
 [5.7        4.4        1.5        0.4       ]
 [5.4        3.9        1.3        0.4       ]
 [5.1        3.5        1.4        0.3       ]
 [5.87714286 3.8        1.7        0.3       ]
 [5.1        3.8        1.5        0.3       ]
 [5.4        3.4        1.7        0.2       ]
 [5.1        

# Regression Imputation/Conditional Mean Imputation¶

In [16]:
from sklearn import linear_model
iris_MAR_regression = iris_MAR.copy()

In [17]:
iris_MAR_regression_model = iris_MAR_regression.copy().dropna()

model = linear_model.LinearRegression()
Xs = iris_MAR_regression_model['sepal width (cm)'].values.reshape(-1, 1)
ys = iris_MAR_regression_model['sepal length (cm)'].values.reshape(-1, 1)
model.fit(X = Xs, y = ys)

null_index = iris_MAR_regression['sepal length (cm)'].isnull()

na_result = model.predict(iris_MAR_regression[null_index]['sepal width (cm)'].values.reshape(-1, 1))


iris_MAR_regression.loc[iris_MAR_regression['sepal length (cm)'].isnull(), 'sepal length (cm)'] = na_result.reshape(len(na_result),)

iris_MAR_regression

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.10000,3.5,1.4,0.2
1,4.90000,3.0,1.4,0.2
2,4.70000,3.2,1.3,0.2
3,4.60000,3.1,1.5,0.2
4,5.00000,3.6,1.4,0.2
...,...,...,...,...
145,6.70000,3.0,5.2,2.3
146,6.30000,2.5,5.0,1.9
147,6.50000,3.0,5.2,2.0
148,5.77987,3.4,5.4,2.3


# Nearest neighbours or hot-deck imputation¶

In [18]:
from sklearn.impute import KNNImputer
iris_MAR_knn = iris_MAR.copy()

In [19]:
knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")

knn_imputer.fit_transform(iris_MAR_knn)

array([[5.1 , 3.5 , 1.4 , 0.2 ],
       [4.9 , 3.  , 1.4 , 0.2 ],
       [4.7 , 3.2 , 1.3 , 0.2 ],
       [4.6 , 3.1 , 1.5 , 0.2 ],
       [5.  , 3.6 , 1.4 , 0.2 ],
       [5.1 , 3.9 , 1.7 , 0.4 ],
       [4.6 , 3.4 , 1.4 , 0.3 ],
       [5.  , 3.4 , 1.5 , 0.2 ],
       [4.85, 2.9 , 1.4 , 0.2 ],
       [4.75, 3.1 , 1.5 , 0.1 ],
       [5.1 , 3.7 , 1.5 , 0.2 ],
       [4.8 , 3.4 , 1.6 , 0.2 ],
       [4.8 , 3.  , 1.4 , 0.1 ],
       [4.3 , 3.  , 1.1 , 0.1 ],
       [5.8 , 4.  , 1.2 , 0.2 ],
       [5.7 , 4.4 , 1.5 , 0.4 ],
       [5.4 , 3.9 , 1.3 , 0.4 ],
       [5.1 , 3.5 , 1.4 , 0.3 ],
       [5.1 , 3.8 , 1.7 , 0.3 ],
       [5.1 , 3.8 , 1.5 , 0.3 ],
       [5.4 , 3.4 , 1.7 , 0.2 ],
       [5.1 , 3.7 , 1.5 , 0.4 ],
       [4.6 , 3.6 , 1.  , 0.2 ],
       [5.  , 3.3 , 1.7 , 0.5 ],
       [5.1 , 3.4 , 1.9 , 0.2 ],
       [5.  , 3.  , 1.6 , 0.2 ],
       [5.  , 3.4 , 1.6 , 0.4 ],
       [5.05, 3.5 , 1.5 , 0.2 ],
       [5.1 , 3.4 , 1.4 , 0.2 ],
       [4.7 , 3.2 , 1.6 , 0.2 ],
       [4.